# Ciência e visualização de dados na Saúde

## Determinação de informações relevantes para classificação de fracasso clínico após cateterismo

### Preparação dos Dados

#### Danielle Lanzarini Pinto      RA: 116559
#### Jampierre Vieira Rocha       RA: 231372
#### Tito Barbosa Rezende         RA: 025327


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pprint
pp = pprint.PrettyPrinter(indent=4)
%matplotlib inline

In [4]:
DATASET_DIR = '../dataset/'

Procedimento para merge dos dados.

As variáveis respostas estão na planilha de Evolução. Esta planilha possui duas colunas de Id ('evolucao_Nº. Paciente' and 'evolucao_Nº da Intervenção')

O merge será feito na seguinte ordem:

evolucao << paciente (on 'evolucao_Nº. Paciente') << procedimento (on 'evolucao_Nº. Paciente' and 'evolucao_Nº da Intervenção') << complicacao (on 'evolucao_Nº. Paciente' and 'evolucao_Nº da Intervenção')


# Cleaning

#### Evolucao

In [5]:
evolucao_df = pd.read_csv(DATASET_DIR+'evolucao.csv')
evolucao_df.drop(columns=['Unnamed: 0'], inplace=True)

In [6]:
# Select all duplicate rows based on one column
duplicateRowsDF = evolucao_df[evolucao_df.duplicated(['evolucao_Nº da Intervenção'], keep=False)]
duplicateRowsDF = duplicateRowsDF.sort_values(by=['evolucao_Nº da Intervenção'])

In [7]:
duplicatedCompositeKeys = (evolucao_df[evolucao_df.duplicated(subset=['evolucao_Nº. Paciente','evolucao_Nº da Intervenção'], keep=False)]).sort_values(by=['evolucao_Nº. Paciente'])

In [9]:
len(evolucao_df[evolucao_df["evolucao_Nº. Paciente"] == 19542])

3

Será preciso remover o paciente 19542 devido a ambiguidade do ID

#### Paciente

In [10]:
paciente_df = pd.read_csv(DATASET_DIR+'paciente.csv')
paciente_df.drop(columns=['Unnamed: 0'], inplace=True)

In [11]:
# Select all duplicate rows based on one column
duplicateRowsDF = paciente_df[paciente_df.duplicated(['paciente_Nº. Paciente'], keep=False)]
duplicateRowsDF = duplicateRowsDF.sort_values(by=['paciente_Nº. Paciente'])
len(duplicateRowsDF)

0

#### Procedimento

In [12]:
procedimento_df = pd.read_csv(DATASET_DIR+'procedimento.csv')
procedimento_df.drop(columns=['Unnamed: 0'], inplace=True)
len(procedimento_df)

9644

In [13]:
# Select all duplicate rows based on one column
duplicateRowsDF = procedimento_df[procedimento_df.duplicated(['procedimento_Nº da Intervenção'], keep=False)]
duplicateRowsDF = duplicateRowsDF.sort_values(by=['procedimento_Nº da Intervenção'])
len(duplicateRowsDF)

692

In [14]:
duplicatedCompositeKeys = (procedimento_df[procedimento_df.duplicated(subset=['procedimento_Nº. Paciente','procedimento_Nº da Intervenção'], keep=False)]).sort_values(by=['procedimento_Nº. Paciente'])
len(duplicatedCompositeKeys)

14

In [15]:
duplicatedCompositeKeys.index

Int64Index([15, 1204, 134, 8600, 1575, 5294, 5359, 7747, 1700, 7814, 1706,
            2938, 664, 7923],
           dtype='int64')

Esses valores nos fizeram perceber que existe mais de um paciente por ID de intervenção, logo a chave precisa ser composta
Essas combinações apresentam mais de uma combinação por paciente e intervenção na tabela de procedimentos e serão excluidos para evitar inconsistencias, dado que não sabemos o motivo

In [16]:
procedimento_df.drop(index=duplicatedCompositeKeys.index, inplace=True, errors='raise')

#### Complicação

In [17]:
complicacao_df = pd.read_csv(DATASET_DIR+'complicacao.csv')
complicacao_df.drop(columns=['Unnamed: 0'], inplace=True)
len(complicacao_df)

1113

In [18]:
# Select all duplicate rows based on one column
duplicateRowsDF = complicacao_df[complicacao_df.duplicated(['complicacao_Nº da Intervenção'], keep=False)]
duplicateRowsDF = duplicateRowsDF.sort_values(by=['complicacao_Nº da Intervenção'])
len(duplicateRowsDF)

167

In [19]:
duplicatedCompositeKeys = (complicacao_df[complicacao_df.duplicated(subset=['complicacao_Nº. Paciente','complicacao_Nº da Intervenção'], keep=False)]).sort_values(by=['complicacao_Nº. Paciente'])
len(duplicatedCompositeKeys)

155

Os dados duplicados serão tratados com agregação (MAX, MIN, SUM, COUNT)

#### Balão

In [21]:
balao_df = pd.read_csv(DATASET_DIR+'balao.csv')
balao_df.drop(columns=['Unnamed: 0'], inplace=True)
len(balao_df)

1175

In [22]:
# Select all duplicate rows based on one column
duplicateRowsDF = balao_df[balao_df.duplicated(['balao_Nº da Intervenção'], keep=False)]
duplicateRowsDF = duplicateRowsDF.sort_values(by=['balao_Nº da Intervenção'])
len(duplicateRowsDF)

218

In [23]:
duplicatedCompositeKeys = (balao_df[balao_df.duplicated(subset=['balao_Nº. Paciente','balao_Nº da Intervenção'], keep=False)]).sort_values(by=['balao_Nº. Paciente'])
len(duplicatedCompositeKeys)

196

Os dados duplicados serão tratados com agregação (MAX, MIN, SUM, COUNT)

#### Stent

In [24]:
stent_df = pd.read_csv(DATASET_DIR+'stent.csv')
stent_df.drop(columns=['Unnamed: 0'], inplace=True)
len(stent_df)

12379

In [25]:
# Select all duplicate rows based on one column
duplicateRowsDF = stent_df[stent_df.duplicated(['stent_Nº da Intervenção'], keep=False)]
duplicateRowsDF = duplicateRowsDF.sort_values(by=['stent_Nº da Intervenção'])
len(duplicateRowsDF)

6284

In [26]:
duplicatedCompositeKeys = (stent_df[stent_df.duplicated(subset=['stent_Nº. Paciente','stent_Nº da Intervenção'], keep=False)]).sort_values(by=['stent_Nº. Paciente'])
len(duplicatedCompositeKeys)

5896

Os dados duplicados serão tratados com agregação (MAX, MIN, SUM, COUNT)

#### Vaso

In [27]:
vaso_df = pd.read_csv(DATASET_DIR+'vaso.csv')
vaso_df.drop(columns=['Unnamed: 0'], inplace=True)
len(vaso_df)

13089

In [28]:
# Select all duplicate rows based on one column
duplicateRowsDF = vaso_df[vaso_df.duplicated(['vaso_Nº da Intervenção'], keep=False)]
duplicateRowsDF = duplicateRowsDF.sort_values(by=['vaso_Nº da Intervenção'])
len(duplicateRowsDF)

6701

In [29]:
duplicatedCompositeKeys = (vaso_df[vaso_df.duplicated(subset=['vaso_Nº. Paciente','vaso_Nº da Intervenção'], keep=False)]).sort_values(by=['vaso_Nº. Paciente'])
len(duplicatedCompositeKeys)

6251

In [30]:
evolucao_df = evolucao_df[evolucao_df["evolucao_Nº. Paciente"] != 19542]
paciente_df = paciente_df[paciente_df["paciente_Nº. Paciente"] != 19542]
procedimento_df = procedimento_df[procedimento_df["procedimento_Nº. Paciente"] != 19542]
complicacao_df = complicacao_df[complicacao_df["complicacao_Nº. Paciente"] != 19542]
balao_df = balao_df[balao_df["balao_Nº. Paciente"] != 19542]
stent_df = stent_df[stent_df["stent_Nº. Paciente"] != 19542]
vaso_df = vaso_df[vaso_df["vaso_Nº. Paciente"] != 19542]

# Transformation

## Evolucao

#### Operations

| Variável                               | Operação                                                    |
|----------------------------------------|-------------------------------------------------------------|
| evolucao_Nº. Paciente                  | CHAVE DE CRUZAMENTO E AGREGADOR                             |
| evolucao_Nº da Intervenção             | CHAVE DE CRUZAMENTO E AGREGADOR                             |
| evolucao_Convênio                      | X                                                           |
| evolucao_Sucesso Clínico               | enconding Sim/Não => 1/0 renomear Problema_desfecho_clinico |
| evolucao_Infarto após Intervenção      | enconding Sim/Não => 1/0                                    |
| evolucao_Tipo Infarto após Intervenção | X                                                           |
| evolucao_Óbito Hospitalar              | enconding Sim/Não => 1/0                                    |
| evolucao_Causa do Óbito Hospitalar     | X                                                           |
| evolucao_Oclusão                       | X                                                           |
| evolucao_Tratamento da Oclusão         | X                                                           |
| evolucao_Complicação Vascular          | enconding Sim/Não => 1/0                                    |
| evolucao_Tipo Complicação Vascular     | X                                                           |
| evolucao_I.R.A                         | enconding Sim/Não => 1/0                                    |
| evolucao_I.R.A Dialítica               | enconding Sim/Não => 1/0                                    |
| evolucao_A.V.C Hemorrágico             | enconding Sim/Não => 1/0                                    |
| evolucao_A.V.C Isquêmico               | enconding Sim/Não => 1/0                                    |
| evolucao_Sangramento TIMI              | X                                                           |
| evolucao_Sangramento GUSTO             | X                                                           |
| evolucao_Quantidade Vaso Tratado       | X                                                           |

In [31]:
evolucao_df.rename(columns={"evolucao_Sucesso Clínico": "evolucao_Problema Clínico"}, inplace=True)
evolucao_df.drop(columns=['evolucao_Convênio','evolucao_Tipo Infarto após Intervenção','evolucao_Causa do Óbito Hospitalar', 'evolucao_Oclusão', 'evolucao_Tratamento da Oclusão', 'evolucao_Tipo Complicação Vascular', 'evolucao_Sangramento TIMI', 'evolucao_Sangramento GUSTO', 'evolucao_Quantidade Vaso Tratado'], inplace=True)

In [32]:
for col in evolucao_df.columns:
    if col in ['evolucao_Nº. Paciente', 'evolucao_Nº da Intervenção']:
        continue
        
    if ('evolucao_Problema Clínico' in col):
        evolucao_df[col] = (evolucao_df[col] == 'Não').astype(int)
    else:
        evolucao_df[col] = (evolucao_df[col] == 'Sim').astype(int)

In [33]:
evolucao_df['evolucao_Combinado'] = evolucao_df.apply (lambda row: int(row[[col for col in evolucao_df.columns if ('evolucao' in col) and ('Paciente' not in col) and ('Intervenção' not in col)]].sum() > 0), axis=1)

In [34]:
evolucao_df.columns

Index(['evolucao_Nº. Paciente', 'evolucao_Nº da Intervenção',
       'evolucao_Problema Clínico', 'evolucao_Infarto após Intervenção',
       'evolucao_Óbito Hospitalar', 'evolucao_Complicação Vascular',
       'evolucao_I.R.A', 'evolucao_I.R.A Dialítica',
       'evolucao_A.V.C Hemorrágico', 'evolucao_A.V.C Isquêmico',
       'evolucao_Combinado'],
      dtype='object')

In [35]:
print('Index length', len(evolucao_df.index))
print('Unique', len(evolucao_df.index.unique()))

Index length 9636
Unique 9636


## Paciente

#### Operations

| Variável                                      | Operação                        |
|-----------------------------------------------|---------------------------------|
| paciente_Nº. Paciente                         | CHAVE DE CRUZAMENTO E AGREGADOR |
| paciente_Sexo                                 | encoding                        |
| paciente_Idade Atual                          | numérica                        |
| paciente_Idade no procedimento                | numérica                        |
| paciente_Raça                                 | encoding                        |
| paciente_Procedência                          | encoding                        |
| paciente_Grau de Instrução                    | encoding                        |
| paciente_Faixa Etária                         | X                               |
| paciente_HAS                                  | encoding                        |
| paciente_Diabetes                             | encoding                        |
| paciente_Tratamento Diabetes                  | encoding                        |
| paciente_Dislipidemia                         | encoding                        |
| paciente_Tratamento Dislipidemia              | encoding                        |
| paciente_Tabagista                            | encoding                        |
| paciente_Cigarros (dia) - Tabagista           | encoding                        |
| paciente_Tempo                                | encoding                        |
| paciente_Ex-Tabagista                         | encoding                        |
| paciente_Cigarros (dia) - Ex-Tabagista        | encoding                        |
| paciente_Tempo de Interrupção                 | encoding                        |
| paciente_IAM Prévio                           | encoding                        |
| paciente_Síndrome Metabólica                  | encoding                        |
| paciente_Antecedentes Familiares              | encoding                        |
| paciente_Tipo Antecedentes Familiares         | encoding                        |
| paciente_IRC                                  | encoding                        |
| paciente_Tipo IRC                             | encoding                        |
| paciente_AVC prévio                           | encoding                        |
| paciente_Doença Vascular Periférica           | encoding                        |
| paciente_Tipo Doença Vascular Periférica      | encoding                        |
| paciente_História de Trombose Venosa Profunda | encoding                        |
| paciente_DPOC                                 | encoding                        |
| paciente_Alergia ao iodo                      | encoding                        |
| paciente_DAC conhecida                        | encoding                        |
| paciente_Cate Prévio                          | encoding                        |
| paciente_Cirurgia de Revascularização Prévia  | encoding                        |
| paciente_Intervenção Percutânea Prévia        | encoding                        |
| paciente_Pressão Arterial                     | 120-130 (split_min_max)         |
| paciente_Frequência Cardíaca                  | numérica                        |
| paciente_Pressão Arterial Diastólica          | 60-80 (split_min_max)           |
| paciente_Frequência Respiratória              | numérica                        |
| paciente_Temperatura (ºC)                     | numérica                        |
| paciente_Cintura Abdominal (cm)               | X                               |
| paciente_Peso (Kg)                            | numérica                        |
| paciente_Altura (metros)                      | numérica                        |
| paciente_Índice de Massa Corporal             | numérica                        |
| paciente_Creatinina Sérica                    | numérica                        |
| paciente_MDRD (ml/min/1,73m²)                 | numérica                        |
| paciente_Clearance Cr (ml/min)                | numérica                        |

Não é necessário agregar pois não há repetição de pacientes

In [36]:
paciente_df.drop(columns=['paciente_Cintura Abdominal (cm)', 'paciente_Faixa Etária'], inplace=True)

In [37]:
new = paciente_df['paciente_Pressão Arterial'].str.split("-", n = 1, expand = True)
new[0] = new[0].str.replace('<','')
new[0] = new[0].str.replace('>','')
new[0] = new[0].str.replace('=','')
new[1] = new[1].str.replace('<','')
new[1] = new[1].str.replace('>','')
new[1] = new[1].str.replace('=','')
paciente_df['paciente_Pressão Arterial_min']= new[0] 
paciente_df['paciente_Pressão Arterial_max']= new[1]

new = paciente_df['paciente_Pressão Arterial Diastólica'].str.split("-", n = 1, expand = True)
new[0] = new[0].str.replace('<','')
new[0] = new[0].str.replace('>','')
new[0] = new[0].str.replace('=','')
new[1] = new[1].str.replace('<','')
new[1] = new[1].str.replace('>','')
new[1] = new[1].str.replace('=','')
paciente_df['paciente_Pressão Arterial Diastólica_min']= new[0] 
paciente_df['paciente_Pressão Arterial Diastólica_max']= new[1]

paciente_df.drop(columns=['paciente_Pressão Arterial', 'paciente_Pressão Arterial Diastólica'], inplace=True)

In [38]:
cols_encoding = ['paciente_Sexo', 'paciente_Raça', 'paciente_Procedência', \
'paciente_Grau de Instrução', 'paciente_HAS', 'paciente_Diabetes', \
'paciente_Tratamento Diabetes', 'paciente_Dislipidemia', 'paciente_Tratamento Dislipidemia', \
'paciente_Tabagista', 'paciente_Cigarros (dia) - Tabagista', 'paciente_Tempo', \
'paciente_Ex-Tabagista', 'paciente_Cigarros (dia) - Ex-Tabagista', 'paciente_Tempo de Interrupção', \
'paciente_IAM Prévio', 'paciente_Síndrome Metabólica', 'paciente_Antecedentes Familiares', \
'paciente_Tipo Antecedentes Familiares', 'paciente_IRC', 'paciente_Tipo IRC', \
'paciente_AVC prévio', 'paciente_Doença Vascular Periférica', 'paciente_Tipo Doença Vascular Periférica', \
'paciente_História de Trombose Venosa Profunda', 'paciente_DPOC', 'paciente_Alergia ao iodo', \
'paciente_DAC conhecida', 'paciente_Cate Prévio', 'paciente_Cirurgia de Revascularização Prévia', \
'paciente_Intervenção Percutânea Prévia']
paciente_df = pd.get_dummies(paciente_df, columns = cols_encoding)

In [39]:
paciente_df.columns

Index(['paciente_Nº. Paciente', 'paciente_Idade Atual',
       'paciente_Idade no procedimento', 'paciente_Frequência Cardíaca',
       'paciente_Frequência Respiratória', 'paciente_Temperatura (ºC)',
       'paciente_Peso (Kg)', 'paciente_Altura (metros)',
       'paciente_Índice de Massa Corporal', 'paciente_Creatinina Sérica',
       'paciente_MDRD (ml/min/1,73m²)', 'paciente_Clearance Cr (ml/min)',
       'paciente_Pressão Arterial_min', 'paciente_Pressão Arterial_max',
       'paciente_Pressão Arterial Diastólica_min',
       'paciente_Pressão Arterial Diastólica_max', 'paciente_Sexo_Feminino',
       'paciente_Sexo_Masculino', 'paciente_Raça_Amarelo',
       'paciente_Raça_Branco', 'paciente_Raça_Mestiço', 'paciente_Raça_Negro',
       'paciente_Procedência_Ambulatorial', 'paciente_Procedência_Internado',
       'paciente_Grau de Instrução_1ª a 4ª série',
       'paciente_Grau de Instrução_2º grau',
       'paciente_Grau de Instrução_5ª a 8ª série',
       'paciente_Grau de Instr

## Procedimento

#### Operations
| Variável                                                  | Operação                                      |
|-----------------------------------------------------------|-----------------------------------------------|
| procedimento_Nº. Paciente                                 | CHAVE DE CRUZAMENTO E AGREGADOR               |
| procedimento_Nº da Intervenção                            | CHAVE DE CRUZAMENTO E AGREGADOR               |
| procedimento_Anestesia - Intervenção                      | enconding e contagem por paciente+intervenção |
| procedimento_Droga - Intervenção                          | enconding e contagem por paciente+intervenção |
| procedimento_Técnica - Intervenção                        | enconding e contagem por paciente+intervenção |
| procedimento_Introdutor - Intervenção                     | enconding e contagem por paciente+intervenção |
| procedimento_Via de acesso - Intervenção                  | enconding e contagem por paciente+intervenção |
| procedimento_Lado Via de Acesso - Intervenção             | enconding e contagem por paciente+intervenção |
| procedimento_Tipo Contraste - Intervenção                 | enconding e contagem por paciente+intervenção |
| procedimento_Nome Comercial - Intervenção                 | enconding e contagem por paciente+intervenção |
| procedimento_Tipo de Intervenção                          | enconding e contagem por paciente+intervenção |
| procedimento_Trombolítico Prévio - Primária               | enconding e contagem por paciente+intervenção |
| procedimento_Trombolítico Prévio - Resgate                | enconding e contagem por paciente+intervenção |
| procedimento_Trombolítico Prévio - Eletiva                | enconding e contagem por paciente+intervenção |
| procedimento_Trombolítico Prévio - Ad-hoc                 | enconding e contagem por paciente+intervenção |
| procedimento_TCA                                          | enconding e contagem por paciente+intervenção |
| procedimento_AAS                                          | enconding e contagem por paciente+intervenção |
| procedimento_Ticlopidina                                  | enconding e contagem por paciente+intervenção |
| procedimento_Clopidogrel                                  | enconding e contagem por paciente+intervenção |
| procedimento_HNF                                          | enconding e contagem por paciente+intervenção |
| procedimento_HBPM                                         | enconding e contagem por paciente+intervenção |
| procedimento_Abciximab                                    | enconding e contagem por paciente+intervenção |
| procedimento_Tirofiban                                    | enconding e contagem por paciente+intervenção |
| procedimento_Eptifibatide                                 | enconding e contagem por paciente+intervenção |
| procedimento_Adenosina                                    | enconding e contagem por paciente+intervenção |
| procedimento_Ultrasom Intracoronário                      | enconding e contagem por paciente+intervenção |
| procedimento_Angiografia Quantitativa                     | enconding e contagem por paciente+intervenção |
| procedimento_Inibidor de glicoproteína                    | enconding e contagem por paciente+intervenção |
| procedimento_Clopidogrel.1                                | enconding e contagem por paciente+intervenção |
| procedimento_Porta/Interv.(hrs) - Urgência                | máx, min, contagem                            |
| procedimento_Dor/ATC (horas) - Resgate                    | máx, min, contagem                            |
| procedimento_Dor/ATC (dias) - Eletiva - Ad-hoc            | máx, min, contagem                            |
| procedimento_Qtde (ml)                                    | min, max, soma, contagem                      |
| procedimento_Duração Interv. (min)                        | soma, contagem media, max, min                |
| procedimento_Dose (mg)                                    | soma, contagem media, max, min                |
| procedimento_Tempo origem/balão (minutos) - Primária      | soma, contagem, media, max, min               |
| procedimento_Tempo dor/balão (minutos) - Primária         | soma, contagem, media, max, min               |
| procedimento_Tempo Hosp. Band./balão (minutos) - Primária | soma, contagem, media, max, min               |
| procedimento_Convênio                                     | X                                             |
| procedimento_Preceptor - Intervenção                      | x                                             |
| procedimento_Operador 1 - Intervenção                     | x                                             |
| procedimento_Operador 2 - Intervenção                     | x                                             |
| procedimento_Ano da Intervenção                           | X                                             |
| procedimento_Data da Intervenção                          | X                                             |

In [40]:
cols_to_drop = ['procedimento_Convênio', 'procedimento_Preceptor - Intervenção', 'procedimento_Operador 1 - Intervenção', \
                'procedimento_Operador 2 - Intervenção', 'procedimento_Ano da Intervenção', 'procedimento_Data da Intervenção']
procedimento_df.drop(columns=cols_to_drop, inplace=True)

In [41]:
cols_encoding = ['procedimento_Anestesia - Intervenção', 'procedimento_Droga - Intervenção', 'procedimento_Técnica - Intervenção',\
                'procedimento_Introdutor - Intervenção', 'procedimento_Via de acesso - Intervenção', \
                'procedimento_Lado Via de Acesso - Intervenção', 'procedimento_Tipo Contraste - Intervenção', \
                'procedimento_Nome Comercial - Intervenção', 'procedimento_Tipo de Intervenção', \
                'procedimento_Trombolítico Prévio - Primária', 'procedimento_Trombolítico Prévio - Resgate', \
                'procedimento_Trombolítico Prévio - Eletiva', 'procedimento_Trombolítico Prévio - Ad-hoc', 'procedimento_TCA', \
                'procedimento_AAS', 'procedimento_Ticlopidina', 'procedimento_Clopidogrel', 'procedimento_HNF', 'procedimento_HBPM',\
                'procedimento_Abciximab', 'procedimento_Tirofiban', 'procedimento_Eptifibatide', 'procedimento_Adenosina', \
                'procedimento_Ultrasom Intracoronário', 'procedimento_Angiografia Quantitativa', 'procedimento_Inibidor de glicoproteína',\
                'procedimento_Clopidogrel.1']
procedimento_df = pd.get_dummies(procedimento_df, columns = cols_encoding)

In [42]:
procedimento_df.columns

Index(['procedimento_Nº. Paciente', 'procedimento_Nº da Intervenção',
       'procedimento_Qtde (ml)', 'procedimento_Porta/Interv.(hrs) - Urgência',
       'procedimento_Dor/ATC (horas) - Resgate',
       'procedimento_Dor/ATC (dias) - Eletiva - Ad-hoc',
       'procedimento_Duração Interv. (min)', 'procedimento_Dose (mg)',
       'procedimento_Tempo origem/balão (minutos) - Primária',
       'procedimento_Tempo dor/balão (minutos) - Primária',
       ...
       'procedimento_Ultrasom Intracoronário_Não',
       'procedimento_Ultrasom Intracoronário_Sim',
       'procedimento_Angiografia Quantitativa_Não',
       'procedimento_Angiografia Quantitativa_Sim',
       'procedimento_Inibidor de glicoproteína_Agrastat',
       'procedimento_Inibidor de glicoproteína_Nenhum',
       'procedimento_Inibidor de glicoproteína_Reo-Pro',
       'procedimento_Clopidogrel.1_< 6h', 'procedimento_Clopidogrel.1_> 6h',
       'procedimento_Clopidogrel.1_Não'],
      dtype='object', length=125)

In [43]:
aggregate_params = {}
for col in procedimento_df.columns:
    print(col)
    if (col in ['procedimento_Porta/Interv.(hrs) - Urgência', 'procedimento_Dor/ATC (horas) - Resgate','procedimento_Dor/ATC (dias) - Eletiva - Ad-hoc']):
        aggregate_params[col] = ['max', 'min', 'count']  #Contagem aqui dá certo?
    elif (col in ['procedimento_Duração Interv. (min)', 'procedimento_Dose (mg)', 'procedimento_Tempo origem/balão (minutos) - Primária', 'procedimento_Tempo dor/balão (minutos) - Primária', 'procedimento_Tempo Hosp. Band./balão (minutos) - Primária']):
        aggregate_params[col] = ['max', 'min', 'mean', 'sum', 'count']
    elif (col in ['procedimento_Qtde (ml)']):
        aggregate_params[col] = ['max', 'min', 'sum', 'count']
    for col_encoding in cols_encoding:
        if (col_encoding in col):
            aggregate_params[col] = ['sum']

procedimento_Nº. Paciente
procedimento_Nº da Intervenção
procedimento_Qtde (ml)
procedimento_Porta/Interv.(hrs) - Urgência
procedimento_Dor/ATC (horas) - Resgate
procedimento_Dor/ATC (dias) - Eletiva - Ad-hoc
procedimento_Duração Interv. (min)
procedimento_Dose (mg)
procedimento_Tempo origem/balão (minutos) - Primária
procedimento_Tempo dor/balão (minutos) - Primária
procedimento_Tempo Hosp. Band./balão (minutos) - Primária
procedimento_Anestesia - Intervenção_Geral Inalatória
procedimento_Anestesia - Intervenção_Geral Sedação
procedimento_Anestesia - Intervenção_Local
procedimento_Droga - Intervenção_Fentanil
procedimento_Droga - Intervenção_Midazolan
procedimento_Droga - Intervenção_Propafenona
procedimento_Droga - Intervenção_Thionembutal
procedimento_Droga - Intervenção_Xilocaína 2%
procedimento_Técnica - Intervenção_Convencional
procedimento_Técnica - Intervenção_kissing-balloon
procedimento_Introdutor - Intervenção_10F
procedimento_Introdutor - Intervenção_11F
procedimento_Introd

In [44]:
pp.pprint(aggregate_params)

{   'procedimento_AAS_Não': ['sum'],
    'procedimento_AAS_Sim': ['sum'],
    'procedimento_Abciximab_Não': ['sum'],
    'procedimento_Abciximab_Sim': ['sum'],
    'procedimento_Adenosina_Não': ['sum'],
    'procedimento_Adenosina_Sim': ['sum'],
    'procedimento_Anestesia - Intervenção_Geral Inalatória': ['sum'],
    'procedimento_Anestesia - Intervenção_Geral Sedação': ['sum'],
    'procedimento_Anestesia - Intervenção_Local': ['sum'],
    'procedimento_Angiografia Quantitativa_Não': ['sum'],
    'procedimento_Angiografia Quantitativa_Sim': ['sum'],
    'procedimento_Clopidogrel.1_< 6h': ['sum'],
    'procedimento_Clopidogrel.1_> 6h': ['sum'],
    'procedimento_Clopidogrel.1_Não': ['sum'],
    'procedimento_Clopidogrel_Não': ['sum'],
    'procedimento_Clopidogrel_Sim': ['sum'],
    'procedimento_Dor/ATC (dias) - Eletiva - Ad-hoc': ['max', 'min', 'count'],
    'procedimento_Dor/ATC (horas) - Resgate': ['max', 'min', 'count'],
    'procedimento_Dose (mg)': ['max', 'min', 'mean', 'sum',

In [45]:
procedimento_agg_df = procedimento_df.groupby(['procedimento_Nº. Paciente','procedimento_Nº da Intervenção']).agg(aggregate_params)
procedimento_agg_df.reset_index(inplace=True)
len(procedimento_agg_df)

9629

In [46]:
print('Index length', len(procedimento_agg_df.index))
print('Unique', len(procedimento_agg_df.index.unique()))

Index length 9629
Unique 9629


## Complicação

#### Operations
| Variável                       | Operação                        |
|--------------------------------|---------------------------------|
| complicacao_Nº. Paciente       | CHAVE DE CRUZAMENTO E AGREGADOR |
| complicacao_Nº da Intervenção  | CHAVE DE CRUZAMENTO E AGREGADOR |
| complicacao_Quando             | enconding e contagem            |
| complicacao_Destino Alta       | enconding e contagem            |
| complicacao_Óbito              | enconding e contagem            |
| complicacao_Tipo Complicação   | enconding e contagem            |
| complicacao_Grau - Complicação | enconding e contagem            |
| complicacao_Conduta            | enconding e contagem            |
| complicacao_Procedimento       | X - sem variabilidade           |
| complicacao_Contato Realizado  | X - sem variabilidade           |
| complicacao_Complicação        | X - sem variabilidade           |

In [47]:
cols_to_drop = ['complicacao_Procedimento', 'complicacao_Contato Realizado', 'complicacao_Complicação']
complicacao_df.drop(columns=cols_to_drop, inplace=True)

In [48]:
cols_encoding = ['complicacao_Quando', 'complicacao_Destino Alta', 'complicacao_Óbito',\
                'complicacao_Tipo Complicação', 'complicacao_Grau - Complicação', \
                'complicacao_Conduta']
complicacao_df = pd.get_dummies(complicacao_df, columns = cols_encoding)

In [49]:
complicacao_df.columns

Index(['complicacao_Nº. Paciente', 'complicacao_Nº da Intervenção',
       'complicacao_Quando_Até 1 semana', 'complicacao_Quando_Até 24 horas',
       'complicacao_Quando_Intra-exame até a alta',
       'complicacao_Quando_Recuperação', 'complicacao_Quando_Sala de exame',
       'complicacao_Destino Alta_Domiciliar',
       'complicacao_Destino Alta_Enfermaria',
       'complicacao_Destino Alta_Hospital Origem', 'complicacao_Óbito_Não',
       'complicacao_Óbito_Sim', 'complicacao_Tipo Complicação_Alérgica',
       'complicacao_Tipo Complicação_Arritmia',
       'complicacao_Tipo Complicação_Congestiva',
       'complicacao_Tipo Complicação_Embólica',
       'complicacao_Tipo Complicação_Hematoma',
       'complicacao_Tipo Complicação_Isquêmica',
       'complicacao_Tipo Complicação_Neurológica',
       'complicacao_Tipo Complicação_R. Contraste',
       'complicacao_Tipo Complicação_Vagal',
       'complicacao_Tipo Complicação_Vascular',
       'complicacao_Grau - Complicação_Grave',

In [50]:
aggregate_params = {}
for col in complicacao_df.columns:
    print(col)
    if (col not in ['complicacao_Nº. Paciente', 'complicacao_Nº da Intervenção']):
        aggregate_params[col] = ['sum']

complicacao_Nº. Paciente
complicacao_Nº da Intervenção
complicacao_Quando_Até 1 semana
complicacao_Quando_Até 24 horas
complicacao_Quando_Intra-exame até a alta
complicacao_Quando_Recuperação
complicacao_Quando_Sala de exame
complicacao_Destino Alta_Domiciliar
complicacao_Destino Alta_Enfermaria
complicacao_Destino Alta_Hospital Origem
complicacao_Óbito_Não
complicacao_Óbito_Sim
complicacao_Tipo Complicação_Alérgica
complicacao_Tipo Complicação_Arritmia
complicacao_Tipo Complicação_Congestiva
complicacao_Tipo Complicação_Embólica
complicacao_Tipo Complicação_Hematoma
complicacao_Tipo Complicação_Isquêmica
complicacao_Tipo Complicação_Neurológica
complicacao_Tipo Complicação_R. Contraste
complicacao_Tipo Complicação_Vagal
complicacao_Tipo Complicação_Vascular
complicacao_Grau - Complicação_Grave
complicacao_Grau - Complicação_Leve
complicacao_Grau - Complicação_Moderado
complicacao_Conduta_Orientação Telefônica
complicacao_Conduta_Retorno para avaliação Médica


In [51]:
pp.pprint(aggregate_params)

{   'complicacao_Conduta_Orientação Telefônica': ['sum'],
    'complicacao_Conduta_Retorno para avaliação Médica': ['sum'],
    'complicacao_Destino Alta_Domiciliar': ['sum'],
    'complicacao_Destino Alta_Enfermaria': ['sum'],
    'complicacao_Destino Alta_Hospital Origem': ['sum'],
    'complicacao_Grau - Complicação_Grave': ['sum'],
    'complicacao_Grau - Complicação_Leve': ['sum'],
    'complicacao_Grau - Complicação_Moderado': ['sum'],
    'complicacao_Quando_Até 1 semana': ['sum'],
    'complicacao_Quando_Até 24 horas': ['sum'],
    'complicacao_Quando_Intra-exame até a alta': ['sum'],
    'complicacao_Quando_Recuperação': ['sum'],
    'complicacao_Quando_Sala de exame': ['sum'],
    'complicacao_Tipo Complicação_Alérgica': ['sum'],
    'complicacao_Tipo Complicação_Arritmia': ['sum'],
    'complicacao_Tipo Complicação_Congestiva': ['sum'],
    'complicacao_Tipo Complicação_Embólica': ['sum'],
    'complicacao_Tipo Complicação_Hematoma': ['sum'],
    'complicacao_Tipo Complicaçã

In [52]:
complicacao_agg_df = complicacao_df.groupby(['complicacao_Nº. Paciente','complicacao_Nº da Intervenção']).agg(aggregate_params)
complicacao_agg_df.reset_index(inplace=True)
complicacao_agg_df.columns

MultiIndex([(                         'complicacao_Nº. Paciente',    ''),
            (                    'complicacao_Nº da Intervenção',    ''),
            (                  'complicacao_Quando_Até 1 semana', 'sum'),
            (                  'complicacao_Quando_Até 24 horas', 'sum'),
            (        'complicacao_Quando_Intra-exame até a alta', 'sum'),
            (                   'complicacao_Quando_Recuperação', 'sum'),
            (                 'complicacao_Quando_Sala de exame', 'sum'),
            (              'complicacao_Destino Alta_Domiciliar', 'sum'),
            (              'complicacao_Destino Alta_Enfermaria', 'sum'),
            (         'complicacao_Destino Alta_Hospital Origem', 'sum'),
            (                            'complicacao_Óbito_Não', 'sum'),
            (                            'complicacao_Óbito_Sim', 'sum'),
            (            'complicacao_Tipo Complicação_Alérgica', 'sum'),
            (            'complicacao_

In [53]:
print('Index length', len(complicacao_agg_df.index))
print('Unique', len(complicacao_agg_df.index.unique()))

Index length 1030
Unique 1030


## Balão

#### Operations
| Variável                                           | Operação                                         |
|----------------------------------------------------|--------------------------------------------------|
| balao_Nº. Paciente                                 | CHAVE DE CRUZAMENTO E AGREGADOR                  |
| balao_Nº da Intervenção                            | CHAVE DE CRUZAMENTO E AGREGADOR                  |
| balao_Kissing wire - Balão                         | completar vazio com não                          |
| balao_Diâmetro Cateter - Balão                     | contagem, média, máx e min por intervenção       |
| balao_Device Adjunto - Balão                       | enconding por categoria - e contagem em cada uma |
| balao_Resultado Angiográfico - Balão               | enconding por categoria - e contagem em cada uma |
| balao_Tipo - Dissecção - Balão                     | enconding por categoria - e contagem em cada uma |
| balao_Classificação da Lesão (ACC/AHA) - Balão     | enconding por categoria - e contagem em cada uma |
| balao_Distúrbio de Fluxo - Balão                   | enconding por categoria - e contagem em cada uma |
| balao_Tipo do Distúrbio de Fluxo - Balão           | enconding por categoria - e contagem em cada uma |
| balao_Tipo No-Reflow                               | enconding por categoria - e contagem em cada uma |
| balao_Vaso coronário - Balão                       | enconding por intervenção                        |
| balao_Kissing balloon - Balão                      | juntar com a categoria anterior                  |
| balao_Comprimento Cateter - Balão                  | média, máx e min por intervenção                 |
| balao_Pressão Final (ATM)                          | média, máx e min por intervenção                 |
| balao_Grau de estenose pós                         | média, máx e min por intervenção                 |
| balao_Timi pós                                     | média, máx e min por intervenção                 |
| balao_Blush Miocárdico                             | média, máx e min por intervenção                 |
| balao_Total Injetado (mg) - Adenosina - Balão      | soma                                             |
| balao_Total Injetado (mg) - Papaverina - Balão     | soma                                             |
| balao_Total Injetado (mg) - Nitroglicerina - Balão | soma                                             |
| balao_Total Injetado (mg) - Monocordil - Balão     | soma                                             |
| balao_Total Injetado (mg) - Nitroprussiato - Balão | soma                                             |
| balao_Total Injetado (mg) - Adrenalina - Balão     | soma                                             |
| balao_N. Injeções - Adenosina - Balão              | soma por intervenção                             |
| balao_N. Injeções - Nitroglicerina - Balão         | soma por intervenção                             |
| balao_N. Injeções - Monocordil - Balão             | soma por intervenção                             |
| balao_N. Injeções - Nitroprussiato - Balão         | soma por intervenção                             |
| balao_N. Injeções - Adrenalina - Balão             | soma por intervenção                             |
| balao_N. Injeções - Papaverina - Balão             | soma por paciente                                |
| balao_Convênio                                     | X                                                |
| balao_Adenosina - Balão                            | X                                                |
| balao_Dose/Injeção (mg) - Adenosina - Balão        | X                                                |
| balao_Papaverina - Balão                           | X                                                |
| balao_Dose/Injeção (mg) - Papaverina - Balão       | X                                                |
| balao_Nitroglicerina - Balão                       | X                                                |
| balao_Dose/Injeção (mg) - Nitroglicerina - Balão   | X                                                |
| balao_Monocordil - Balão                           | X                                                |
| balao_Dose/Injeção (mg) - Monocordil - Balão       | X                                                |
| balao_Nitroprussiato - Balão                       | X                                                |
| balao_Dose/Injeção (mg) - Nitroprussiato - Balão   | X                                                |
| balao_Adrenalina - Balão                           | X                                                |
| balao_Dose/Injeção (mg) - Adrenalina - Balão       | X                                                |

In [54]:
cols_to_drop = ['balao_Convênio', 'balao_Adenosina - Balão', 'balao_Dose/Injeção (mg) - Adenosina - Balão', \
                'balao_Papaverina - Balão', 'balao_Dose/Injeção (mg) - Papaverina - Balão', 'balao_Nitroglicerina - Balão', \
                'balao_Monocordil - Balão', 'balao_Dose/Injeção (mg) - Monocordil - Balão', 'balao_Nitroprussiato - Balão', \
                'balao_Dose/Injeção (mg) - Nitroprussiato - Balão', 'balao_Adrenalina - Balão', 'balao_Dose/Injeção (mg) - Adrenalina - Balão']
balao_df.drop(columns=cols_to_drop, inplace=True)

In [55]:
cols_encoding = ['balao_Device Adjunto - Balão', 'balao_Resultado Angiográfico - Balão', 'balao_Tipo - Dissecção - Balão', \
                 'balao_Classificação da Lesão (ACC/AHA) - Balão', 'balao_Distúrbio de Fluxo - Balão', 'balao_Tipo do Distúrbio de Fluxo - Balão', \
                 'balao_Tipo No-Reflow', 'balao_Vaso coronário - Balão', 'balao_Kissing wire - Balão', 'balao_Kissing balloon - Balão']
balao_df = pd.get_dummies(balao_df, columns = cols_encoding)

In [56]:
balao_df.columns

Index(['balao_Nº. Paciente', 'balao_Nº da Intervenção',
       'balao_Diâmetro Cateter - Balão', 'balao_Comprimento Cateter - Balão',
       'balao_Pressão Final (ATM)', 'balao_Grau de estenose pós',
       'balao_Timi pós', 'balao_Blush Miocárdico',
       'balao_N. Injeções - Adenosina - Balão',
       'balao_Total Injetado (mg) - Adenosina - Balão',
       'balao_N. Injeções - Papaverina - Balão',
       'balao_Total Injetado (mg) - Papaverina - Balão',
       'balao_N. Injeções - Nitroglicerina - Balão',
       'balao_Dose/Injeção (mg) - Nitroglicerina - Balão',
       'balao_Total Injetado (mg) - Nitroglicerina - Balão',
       'balao_N. Injeções - Monocordil - Balão',
       'balao_Total Injetado (mg) - Monocordil - Balão',
       'balao_N. Injeções - Nitroprussiato - Balão',
       'balao_Total Injetado (mg) - Nitroprussiato - Balão',
       'balao_N. Injeções - Adrenalina - Balão',
       'balao_Total Injetado (mg) - Adrenalina - Balão',
       'balao_Device Adjunto - Balão_Bra

In [57]:
aggregate_params = {}
for col in balao_df.columns:
    print(col)
    if (col in ['balao_Diâmetro Cateter - Balão']):
        aggregate_params[col] = ['count', 'mean', 'max', 'min']
    elif (col in ['balao_Comprimento Cateter - Balão', 'balao_Pressão Final (ATM)', 'balao_Grau de estenose pós', 'balao_Timi pós', 'balao_Blush Miocárdico']):
        aggregate_params[col] = ['mean', 'max', 'min']
    elif (col in ['balao_Total Injetado (mg) - Adenosina - Balão', 'balao_Total Injetado (mg) - Papaverina - Balão', 'balao_Total Injetado (mg) - Nitroglicerina - Balão', 'balao_Total Injetado (mg) - Monocordil - Balão', 'balao_Total Injetado (mg) - Nitroprussiato - Balão', 'balao_Total Injetado (mg) - Adrenalina - Balão']):
        aggregate_params[col] = ['sum']
    elif (col in ['balao_N. Injeções - Adenosina - Balão', 'balao_N. Injeções - Nitroglicerina - Balão', 'balao_N. Injeções - Monocordil - Balão', 'balao_N. Injeções - Nitroprussiato - Balão', 'balao_N. Injeções - Adrenalina - Balão', 'balao_N. Injeções - Papaverina - Balão']):
        aggregate_params[col] = ['sum']
    for col_encoding in cols_encoding:
        if (col_encoding in col):
            aggregate_params[col] = ['sum']

balao_Nº. Paciente
balao_Nº da Intervenção
balao_Diâmetro Cateter - Balão
balao_Comprimento Cateter - Balão
balao_Pressão Final (ATM)
balao_Grau de estenose pós
balao_Timi pós
balao_Blush Miocárdico
balao_N. Injeções - Adenosina - Balão
balao_Total Injetado (mg) - Adenosina - Balão
balao_N. Injeções - Papaverina - Balão
balao_Total Injetado (mg) - Papaverina - Balão
balao_N. Injeções - Nitroglicerina - Balão
balao_Dose/Injeção (mg) - Nitroglicerina - Balão
balao_Total Injetado (mg) - Nitroglicerina - Balão
balao_N. Injeções - Monocordil - Balão
balao_Total Injetado (mg) - Monocordil - Balão
balao_N. Injeções - Nitroprussiato - Balão
balao_Total Injetado (mg) - Nitroprussiato - Balão
balao_N. Injeções - Adrenalina - Balão
balao_Total Injetado (mg) - Adrenalina - Balão
balao_Device Adjunto - Balão_Braquiterapia
balao_Device Adjunto - Balão_Exciser
balao_Resultado Angiográfico - Balão_I1 - Não ultrapassou a lesão
balao_Resultado Angiográfico - Balão_I2 - Ultrapassou e não dilatou
balao_Re

In [58]:
pp.pprint(aggregate_params)

{   'balao_Blush Miocárdico': ['mean', 'max', 'min'],
    'balao_Classificação da Lesão (ACC/AHA) - Balão_A': ['sum'],
    'balao_Classificação da Lesão (ACC/AHA) - Balão_B1': ['sum'],
    'balao_Classificação da Lesão (ACC/AHA) - Balão_B2': ['sum'],
    'balao_Classificação da Lesão (ACC/AHA) - Balão_C': ['sum'],
    'balao_Comprimento Cateter - Balão': ['mean', 'max', 'min'],
    'balao_Device Adjunto - Balão_Braquiterapia': ['sum'],
    'balao_Device Adjunto - Balão_Exciser': ['sum'],
    'balao_Distúrbio de Fluxo - Balão_Não': ['sum'],
    'balao_Distúrbio de Fluxo - Balão_Sim': ['sum'],
    'balao_Diâmetro Cateter - Balão': ['count', 'mean', 'max', 'min'],
    'balao_Grau de estenose pós': ['mean', 'max', 'min'],
    'balao_Kissing balloon - Balão_Não': ['sum'],
    'balao_Kissing balloon - Balão_Sim': ['sum'],
    'balao_Kissing wire - Balão_Não': ['sum'],
    'balao_Kissing wire - Balão_Sim': ['sum'],
    'balao_N. Injeções - Adenosina - Balão': ['sum'],
    'balao_N. Injeções -

In [59]:
balao_agg_df = balao_df.groupby(['balao_Nº. Paciente','balao_Nº da Intervenção']).agg(aggregate_params)
balao_agg_df.reset_index(inplace=True)
balao_agg_df.columns

MultiIndex([(                                                 'balao_Nº. Paciente', ...),
            (                                            'balao_Nº da Intervenção', ...),
            (                                     'balao_Diâmetro Cateter - Balão', ...),
            (                                     'balao_Diâmetro Cateter - Balão', ...),
            (                                     'balao_Diâmetro Cateter - Balão', ...),
            (                                     'balao_Diâmetro Cateter - Balão', ...),
            (                                  'balao_Comprimento Cateter - Balão', ...),
            (                                  'balao_Comprimento Cateter - Balão', ...),
            (                                  'balao_Comprimento Cateter - Balão', ...),
            (                                          'balao_Pressão Final (ATM)', ...),
            (                                          'balao_Pressão Final (ATM)', ...),
          

In [60]:
print('Index length', len(balao_agg_df.index))
print('Unique', len(balao_agg_df.index.unique()))

Index length 1069
Unique 1069


## Stent

#### Operations
| Variável                                           | Operação                                      |
|----------------------------------------------------|-----------------------------------------------|
| stent_Nº. Paciente                                 | CHAVE DE CRUZAMENTO E AGREGADOR               |
| stent_Nº da Intervenção                            | CHAVE DE CRUZAMENTO E AGREGADOR               |
| stent_Vaso coronário - Stent                       | enconding e contagem por paciente+intervenção |
| stent_Stent                                        | enconding e contagem por paciente+intervenção |
| stent_Estratégia                                   | enconding e contagem por paciente+intervenção |
| stent_Resultado Angiográfico - Stent               | enconding e contagem por paciente+intervenção |
| stent_Tipo - Dissecção - Stent                     | enconding e contagem por paciente+intervenção |
| stent_Localização da Dissecção                     | enconding e contagem por paciente+intervenção |
| stent_Classificação da Lesão (ACC/AHA) - Stent     | enconding e contagem por paciente+intervenção |
| stent_Kissing wire - Stent                         | enconding e contagem por paciente+intervenção |
| stent_Kissing balloon - Stent                      | enconding e contagem por paciente+intervenção |
| stent_Sobreposição                                 | enconding e contagem por paciente+intervenção |
| stent_Distúrbio de Fluxo - Stent                   | enconding e contagem por paciente+intervenção |
| stent_Tipo do Distúrbio de Fluxo - Stent           | enconding e contagem por paciente+intervenção |
| stent_Tipo No-Reflow                               | enconding e contagem por paciente+intervenção |
| stent_Técnica de bifurcação                        | enconding e contagem por paciente+intervenção |
| stent_Stent Dedicado                               | enconding e contagem por paciente+intervenção |
| stent_Adenosina - Stent                            | enconding e contagem por paciente+intervenção |
| stent_Papaverina - Stent                           | enconding e contagem por paciente+intervenção |
| stent_Nitroglicerina - Stent                       | enconding e contagem por paciente+intervenção |
| stent_Monocordil - Stent                           | enconding e contagem por paciente+intervenção |
| stent_Nitroprussiato - Stent                       | enconding e contagem por paciente+intervenção |
| stent_Adrenalina - Stent                           | enconding e contagem por paciente+intervenção |
| stent_Comprimento Dispositivo - Stent              | media, max, min                               |
| stent_Diâmetro Cateter Balão Adjunto Pré           | media, max, min                               |
| stent_Diâmetro Cateter Balão Adjunto Pós           | media, max, min                               |
| stent_Pressão Final de Liberação (ATM)             | media, max, min                               |
| stent_Grau de estenose pós                         | media, max, min                               |
| stent_Timi pós                                     | media, max, min                               |
| stent_Blush Miocárdico                             | media, max, min                               |
| stent_Diâmetro Dispositivo - Stent                 | media, max, min, contagem                     |
| stent_N. Injeções - Adenosina - Stent              | soma                                          |
| stent_Total Injetado (mg) - Adenosina - Stent      | soma                                          |
| stent_N. Injeções - Papaverina - Stent             | soma                                          |
| stent_Total Injetado (mg) - Papaverina - Stent     | soma                                          |
| stent_N. Injeções - Nitroglicerina - Stent         | soma                                          |
| stent_Total Injetado (mg) - Nitroglicerina - Stent | soma                                          |
| stent_N. Injeções - Monocordil - Stent             | soma                                          |
| stent_Total Injetado (mg) - Monocordil - Stent     | soma                                          |
| stent_N. Injeções - Nitroprussiato - Stent         | soma                                          |
| stent_Total Injetado (mg) - Nitroprussiato - Stent | soma                                          |
| stent_N. Injeções - Adrenalina - Stent             | soma                                          |
| stent_Total Injetado (mg) - Adrenalina - Stent     | soma                                          |
| stent_Convênio                                     | x                                             |
| stent_Dose/Injeção (mg) - Adenosina - Stent        | X                                             |
| stent_Dose/Injeção (mg) - Papaverina - Stent       | X                                             |
| stent_Dose/Injeção (mg) - Nitroglicerina - Stent   | X                                             |
| stent_Dose/Injeção (mg) - Monocordil - Stent       | X                                             |
| stent_Dose/Injeção (mg) - Nitroprussiato - Stent   | X                                             |
| stent_Dose/Injeção (mg) - Adrenalina - Stent       | X                                             |

In [61]:
cols_to_drop = ['stent_Convênio', 'stent_Dose/Injeção (mg) - Adenosina - Stent', 'stent_Dose/Injeção (mg) - Papaverina - Stent', \
                'stent_Dose/Injeção (mg) - Nitroglicerina - Stent', 'stent_Dose/Injeção (mg) - Monocordil - Stent', \
                'stent_Dose/Injeção (mg) - Nitroprussiato - Stent', 'stent_Dose/Injeção (mg) - Adrenalina - Stent']
stent_df.drop(columns=cols_to_drop, inplace=True)

In [62]:
cols_encoding = ['stent_Vaso coronário - Stent', 'stent_Stent', 'stent_Estratégia', \
                 'stent_Resultado Angiográfico - Stent', 'stent_Tipo - Dissecção - Stent', 'stent_Localização da Dissecção', \
                 'stent_Classificação da Lesão (ACC/AHA) - Stent', 'stent_Kissing wire - Stent', 'stent_Kissing balloon - Stent', \
                 'stent_Sobreposição', 'stent_Distúrbio de Fluxo - Stent', 'stent_Tipo do Distúrbio de Fluxo - Stent', \
                 'stent_Tipo No-Reflow', 'stent_Técnica de bifurcação', 'stent_Stent Dedicado', 'stent_Adenosina - Stent', \
                 'stent_Papaverina - Stent', 'stent_Nitroglicerina - Stent', 'stent_Monocordil - Stent', 'stent_Nitroprussiato - Stent', \
                 'stent_Adrenalina - Stent']
stent_df = pd.get_dummies(stent_df, columns = cols_encoding)

In [64]:
stent_df.columns

Index(['stent_Nº. Paciente', 'stent_Nº da Intervenção',
       'stent_Diâmetro Dispositivo - Stent',
       'stent_Comprimento Dispositivo - Stent',
       'stent_Diâmetro Cateter Balão Adjunto Pré',
       'stent_Diâmetro Cateter Balão Adjunto Pós',
       'stent_Pressão Final de Liberação (ATM)', 'stent_Grau de estenose pós',
       'stent_Timi pós', 'stent_Blush Miocárdico',
       ...
       'stent_Adenosina - Stent_Sim', 'stent_Papaverina - Stent_Não',
       'stent_Papaverina - Stent_Sim', 'stent_Nitroglicerina - Stent_Não',
       'stent_Nitroglicerina - Stent_Sim', 'stent_Monocordil - Stent_Não',
       'stent_Monocordil - Stent_Sim', 'stent_Nitroprussiato - Stent_Não',
       'stent_Adrenalina - Stent_Não', 'stent_Adrenalina - Stent_Sim'],
      dtype='object', length=166)

In [65]:
aggregate_params = {}
for col in stent_df.columns:
    print(col)
    if (col in ['stent_Comprimento Dispositivo - Stent', 'stent_Diâmetro Cateter Balão Adjunto Pré', 'stent_Diâmetro Cateter Balão Adjunto Pós', 'stent_Pressão Final de Liberação (ATM)', 'stent_Grau de estenose pós', 'stent_Timi pós', 'stent_Blush Miocárdico']):
        aggregate_params[col] = ['mean', 'max', 'min']
    elif (col in ['stent_Diâmetro Dispositivo - Stent']):
        aggregate_params[col] = ['mean', 'max', 'min', 'count']
    elif (col in ['stent_N. Injeções - Adenosina - Stent', 'stent_Total Injetado (mg) - Adenosina - Stent', 'stent_N. Injeções - Papaverina - Stent', 'stent_Total Injetado (mg) - Papaverina - Stent', 'stent_N. Injeções - Nitroglicerina - Stent', 'stent_Total Injetado (mg) - Nitroglicerina - Stent', 'stent_N. Injeções - Monocordil - Stent', 'stent_Total Injetado (mg) - Monocordil - Stent', 'stent_N. Injeções - Nitroprussiato - Stent', 'stent_Total Injetado (mg) - Nitroprussiato - Stent', 'stent_N. Injeções - Adrenalina - Stent', 'stent_Total Injetado (mg) - Adrenalina - Stent']):
        aggregate_params[col] = ['sum']
    for col_encoding in cols_encoding:
        if (col_encoding in col):
            aggregate_params[col] = ['sum']

stent_Nº. Paciente
stent_Nº da Intervenção
stent_Diâmetro Dispositivo - Stent
stent_Comprimento Dispositivo - Stent
stent_Diâmetro Cateter Balão Adjunto Pré
stent_Diâmetro Cateter Balão Adjunto Pós
stent_Pressão Final de Liberação (ATM)
stent_Grau de estenose pós
stent_Timi pós
stent_Blush Miocárdico
stent_N. Injeções - Adenosina - Stent
stent_Total Injetado (mg) - Adenosina - Stent
stent_N. Injeções - Papaverina - Stent
stent_Total Injetado (mg) - Papaverina - Stent
stent_N. Injeções - Nitroglicerina - Stent
stent_Total Injetado (mg) - Nitroglicerina - Stent
stent_N. Injeções - Monocordil - Stent
stent_Total Injetado (mg) - Monocordil - Stent
stent_N. Injeções - Nitroprussiato - Stent
stent_Total Injetado (mg) - Nitroprussiato - Stent
stent_N. Injeções - Adrenalina - Stent
stent_Total Injetado (mg) - Adrenalina - Stent
stent_Vaso coronário - Stent_CD
stent_Vaso coronário - Stent_CX
stent_Vaso coronário - Stent_DA
stent_Vaso coronário - Stent_DPD
stent_Vaso coronário - Stent_DPE
stent_

In [66]:
pp.pprint(aggregate_params)

{   'stent_Adenosina - Stent_Não': ['sum'],
    'stent_Adenosina - Stent_Sim': ['sum'],
    'stent_Adrenalina - Stent_Não': ['sum'],
    'stent_Adrenalina - Stent_Sim': ['sum'],
    'stent_Blush Miocárdico': ['mean', 'max', 'min'],
    'stent_Classificação da Lesão (ACC/AHA) - Stent_A': ['sum'],
    'stent_Classificação da Lesão (ACC/AHA) - Stent_B1': ['sum'],
    'stent_Classificação da Lesão (ACC/AHA) - Stent_B2': ['sum'],
    'stent_Classificação da Lesão (ACC/AHA) - Stent_C': ['sum'],
    'stent_Comprimento Dispositivo - Stent': ['mean', 'max', 'min'],
    'stent_Distúrbio de Fluxo - Stent_Não': ['sum'],
    'stent_Distúrbio de Fluxo - Stent_Sim': ['sum'],
    'stent_Diâmetro Cateter Balão Adjunto Pré': ['mean', 'max', 'min'],
    'stent_Diâmetro Cateter Balão Adjunto Pós': ['mean', 'max', 'min'],
    'stent_Diâmetro Dispositivo - Stent': ['mean', 'max', 'min', 'count'],
    'stent_Estratégia_Pré e Pós dilatação': ['sum'],
    'stent_Estratégia_Pré-dilatação': ['sum'],
    'stent_E

In [67]:
stent_agg_df = stent_df.groupby(['stent_Nº. Paciente','stent_Nº da Intervenção']).agg(aggregate_params)
stent_agg_df.reset_index(inplace=True)
stent_agg_df.columns

MultiIndex([(                      'stent_Nº. Paciente',      ''),
            (                 'stent_Nº da Intervenção',      ''),
            (      'stent_Diâmetro Dispositivo - Stent',  'mean'),
            (      'stent_Diâmetro Dispositivo - Stent',   'max'),
            (      'stent_Diâmetro Dispositivo - Stent',   'min'),
            (      'stent_Diâmetro Dispositivo - Stent', 'count'),
            (   'stent_Comprimento Dispositivo - Stent',  'mean'),
            (   'stent_Comprimento Dispositivo - Stent',   'max'),
            (   'stent_Comprimento Dispositivo - Stent',   'min'),
            ('stent_Diâmetro Cateter Balão Adjunto Pré',  'mean'),
            ...
            (             'stent_Adenosina - Stent_Sim',   'sum'),
            (            'stent_Papaverina - Stent_Não',   'sum'),
            (            'stent_Papaverina - Stent_Sim',   'sum'),
            (        'stent_Nitroglicerina - Stent_Não',   'sum'),
            (        'stent_Nitroglicerina - S

In [68]:
print('Index length', len(stent_agg_df.index))
print('Unique', len(stent_agg_df.index.unique()))

Index length 9145
Unique 9145


## Vaso

#### Operations
| Variável                                      | Operação                                                                     |
|-----------------------------------------------|------------------------------------------------------------------------------|
| vaso_Nº. Paciente                             | CHAVE DE CRUZAMENTO E AGREGADOR                                              |
| vaso_Nº da Intervenção                        | CHAVE DE CRUZAMENTO E AGREGADOR                                              |
| vaso_Anatomia - Intervenção                   | enconding e contagem por paciente+intervenção                                |
| vaso_Vaso coronário - Intervenção             | enconding e contagem por paciente+intervenção                                |
| vaso_Local da Lesão - Intervenção             | enconding e contagem por paciente+intervenção                                |
| vaso_Tipo de Lesão - Intervenção              | enconding e contagem por paciente+intervenção                                |
| vaso_Tipo de Lesão - Intrastent - Intervenção | enconding e contagem por paciente+intervenção                                |
| vaso_Tipo de Lesão - Focal - Intervenção      | enconding e contagem por paciente+intervenção                                |
| vaso_Tipo de Lesão - Difusa - Intervenção     | enconding e contagem por paciente+intervenção                                |
| vaso_Tipo de Lesão - Pós balão - Intervenção  | enconding e contagem por paciente+intervenção                                |
| vaso_Calcificação - Intervenção               | enconding e contagem por paciente+intervenção                                |
| vaso_Tortuosidade - Intervenção               | enconding e contagem por paciente+intervenção                                |
| vaso_Angulação - Intervenção                  | enconding e contagem por paciente+intervenção                                |
| vaso_Grau de Importância - Intervenção        | enconding e contagem por paciente+intervenção                                |
| vaso_Maior que 20mm - Intervenção             | enconding e contagem por paciente+intervenção                                |
| vaso_Trombo - Intervenção                     | enconding e contagem por paciente+intervenção                                |
| vaso_Placa Rota - Intervenção                 | enconding e contagem por paciente+intervenção                                |
| vaso_Placa Ulcerada - Intervenção             | enconding e contagem por paciente+intervenção                                |
| vaso_Ectasia - Intervenção                    | enconding e contagem por paciente+intervenção                                |
| vaso_Aneurisma - Intervenção                  | enconding e contagem por paciente+intervenção                                |
| vaso_Bifurcação - Intervenção                 | fazer o enconding mas verificar antes se estão presentes as 8 posssibiidades |
| vaso_Grau de estenose pré - Intervenção       | máx, min, media, soma                                                        |
| vaso_Convênio                                 | x                                                                            |

In [69]:
cols_to_drop = ['vaso_Convênio']
vaso_df.drop(columns=cols_to_drop, inplace=True)

In [70]:
cols_encoding = ['vaso_Anatomia - Intervenção', 'vaso_Vaso coronário - Intervenção', \
                 'vaso_Local da Lesão - Intervenção', 'vaso_Tipo de Lesão - Intervenção', \
                 'vaso_Tipo de Lesão - Intrastent - Intervenção', 'vaso_Tipo de Lesão - Focal - Intervenção', \
                 'vaso_Tipo de Lesão - Difusa - Intervenção', 'vaso_Tipo de Lesão - Pós balão - Intervenção', \
                 'vaso_Calcificação - Intervenção', 'vaso_Tortuosidade - Intervenção', 'vaso_Angulação - Intervenção', \
                 'vaso_Grau de Importância - Intervenção', 'vaso_Maior que 20mm - Intervenção', \
                 'vaso_Trombo - Intervenção', 'vaso_Placa Rota - Intervenção', 'vaso_Placa Ulcerada - Intervenção', \
                 'vaso_Ectasia - Intervenção', 'vaso_Aneurisma - Intervenção', 'vaso_Bifurcação - Intervenção']
vaso_df = pd.get_dummies(vaso_df, columns = cols_encoding)

In [71]:
vaso_df.columns

Index(['vaso_Nº. Paciente', 'vaso_Nº da Intervenção',
       'vaso_Grau de estenose pré - Intervenção',
       'vaso_Anatomia - Intervenção_Ateromatose moderada',
       'vaso_Anatomia - Intervenção_Ateromatose severa',
       'vaso_Anatomia - Intervenção_Com irregularidades parietais',
       'vaso_Anatomia - Intervenção_Com lesão',
       'vaso_Vaso coronário - Intervenção_CD',
       'vaso_Vaso coronário - Intervenção_CX',
       'vaso_Vaso coronário - Intervenção_DA',
       ...
       'vaso_Ectasia - Intervenção_Sim', 'vaso_Aneurisma - Intervenção_Não',
       'vaso_Aneurisma - Intervenção_Sim',
       'vaso_Bifurcação - Intervenção_0,0,1',
       'vaso_Bifurcação - Intervenção_0,1,0',
       'vaso_Bifurcação - Intervenção_0,1,1',
       'vaso_Bifurcação - Intervenção_1,0,0',
       'vaso_Bifurcação - Intervenção_1,0,1',
       'vaso_Bifurcação - Intervenção_1,1,0',
       'vaso_Bifurcação - Intervenção_1,1,1'],
      dtype='object', length=106)

In [72]:
aggregate_params = {}
for col in vaso_df.columns:
    print(col)
    if (col in ['vaso_Grau de estenose pré - Intervenção']):
        aggregate_params[col] = ['max', 'min', 'mean', 'sum']
    for col_encoding in cols_encoding:
        if (col_encoding in col):
            aggregate_params[col] = ['sum']

vaso_Nº. Paciente
vaso_Nº da Intervenção
vaso_Grau de estenose pré - Intervenção
vaso_Anatomia - Intervenção_Ateromatose moderada
vaso_Anatomia - Intervenção_Ateromatose severa
vaso_Anatomia - Intervenção_Com irregularidades parietais
vaso_Anatomia - Intervenção_Com lesão
vaso_Vaso coronário - Intervenção_CD
vaso_Vaso coronário - Intervenção_CX
vaso_Vaso coronário - Intervenção_DA
vaso_Vaso coronário - Intervenção_DPD
vaso_Vaso coronário - Intervenção_DPE
vaso_Vaso coronário - Intervenção_Dg1
vaso_Vaso coronário - Intervenção_Dg2
vaso_Vaso coronário - Intervenção_Dg3
vaso_Vaso coronário - Intervenção_Intermédio-RI
vaso_Vaso coronário - Intervenção_MamáriaCD
vaso_Vaso coronário - Intervenção_MamáriaDA
vaso_Vaso coronário - Intervenção_MamáriaDg
vaso_Vaso coronário - Intervenção_MamáriaMg
vaso_Vaso coronário - Intervenção_Mg1
vaso_Vaso coronário - Intervenção_Mg2
vaso_Vaso coronário - Intervenção_Mg3
vaso_Vaso coronário - Intervenção_Mg4
vaso_Vaso coronário - Intervenção_MgD
vaso_Vaso co

In [73]:
pp.pprint(aggregate_params)

{   'vaso_Anatomia - Intervenção_Ateromatose moderada': ['sum'],
    'vaso_Anatomia - Intervenção_Ateromatose severa': ['sum'],
    'vaso_Anatomia - Intervenção_Com irregularidades parietais': ['sum'],
    'vaso_Anatomia - Intervenção_Com lesão': ['sum'],
    'vaso_Aneurisma - Intervenção_Não': ['sum'],
    'vaso_Aneurisma - Intervenção_Sim': ['sum'],
    'vaso_Angulação - Intervenção_>45': ['sum'],
    'vaso_Angulação - Intervenção_>90': ['sum'],
    'vaso_Angulação - Intervenção_Acentuada': ['sum'],
    'vaso_Angulação - Intervenção_Leve': ['sum'],
    'vaso_Angulação - Intervenção_Moderada': ['sum'],
    'vaso_Bifurcação - Intervenção_0,0,1': ['sum'],
    'vaso_Bifurcação - Intervenção_0,1,0': ['sum'],
    'vaso_Bifurcação - Intervenção_0,1,1': ['sum'],
    'vaso_Bifurcação - Intervenção_1,0,0': ['sum'],
    'vaso_Bifurcação - Intervenção_1,0,1': ['sum'],
    'vaso_Bifurcação - Intervenção_1,1,0': ['sum'],
    'vaso_Bifurcação - Intervenção_1,1,1': ['sum'],
    'vaso_Calcificação - 

In [74]:
vaso_agg_df = vaso_df.groupby(['vaso_Nº. Paciente','vaso_Nº da Intervenção']).agg(aggregate_params)
vaso_agg_df.reset_index(inplace=True)
vaso_agg_df.columns

MultiIndex([(                                        'vaso_Nº. Paciente', ...),
            (                                   'vaso_Nº da Intervenção', ...),
            (                  'vaso_Grau de estenose pré - Intervenção', ...),
            (                  'vaso_Grau de estenose pré - Intervenção', ...),
            (                  'vaso_Grau de estenose pré - Intervenção', ...),
            (                  'vaso_Grau de estenose pré - Intervenção', ...),
            (         'vaso_Anatomia - Intervenção_Ateromatose moderada', ...),
            (           'vaso_Anatomia - Intervenção_Ateromatose severa', ...),
            ('vaso_Anatomia - Intervenção_Com irregularidades parietais', ...),
            (                    'vaso_Anatomia - Intervenção_Com lesão', ...),
            ...
            (                           'vaso_Ectasia - Intervenção_Sim', ...),
            (                         'vaso_Aneurisma - Intervenção_Não', ...),
            (           

In [75]:
print('Index length', len(vaso_agg_df.index))
print('Unique', len(vaso_agg_df.index.unique()))

Index length 9634
Unique 9634


# Merging

In [76]:
def rename_columns(df):
    rename = {}
    for col in df.columns:
        new_col_name = col
        #rename tuples
        if len(col)==2:
            if col[1]=='':
                new_col_name = col[0]
            else:
                new_col_name = col[0]+'_'+col[1]
        #rename replacing chars [,]<>
        new_col_name = new_col_name.replace('[','').replace(']','').replace(',','').replace('<','lt').replace('>','gt')
        print('New col name', new_col_name)
        rename[col]=new_col_name
    return rename

In [77]:
merge_df = pd.merge(evolucao_df, paciente_df,  how='left', left_on=['evolucao_Nº. Paciente'], right_on = ['paciente_Nº. Paciente'])
merge_df.rename(columns=rename_columns(merge_df), inplace=True)

New col name evolucao_Nº. Paciente
New col name evolucao_Nº da Intervenção
New col name evolucao_Problema Clínico
New col name evolucao_Infarto após Intervenção
New col name evolucao_Óbito Hospitalar
New col name evolucao_Complicação Vascular
New col name evolucao_I.R.A
New col name evolucao_I.R.A Dialítica
New col name evolucao_A.V.C Hemorrágico
New col name evolucao_A.V.C Isquêmico
New col name evolucao_Combinado
New col name paciente_Nº. Paciente
New col name paciente_Idade Atual
New col name paciente_Idade no procedimento
New col name paciente_Frequência Cardíaca
New col name paciente_Frequência Respiratória
New col name paciente_Temperatura (ºC)
New col name paciente_Peso (Kg)
New col name paciente_Altura (metros)
New col name paciente_Índice de Massa Corporal
New col name paciente_Creatinina Sérica
New col name paciente_MDRD (ml/min/173m²)
New col name paciente_Clearance Cr (ml/min)
New col name paciente_Pressão Arterial_min
New col name paciente_Pressão Arterial_max
New col name

In [78]:
merge_df = pd.merge(merge_df, procedimento_agg_df,  how='left', left_on=['evolucao_Nº. Paciente','evolucao_Nº da Intervenção'], right_on = ['procedimento_Nº. Paciente','procedimento_Nº da Intervenção'])
merge_df.rename(columns=rename_columns(merge_df), inplace=True)

New col name evolucao_Nº. Paciente
New col name evolucao_Nº da Intervenção
New col name evolucao_Problema Clínico
New col name evolucao_Infarto após Intervenção
New col name evolucao_Óbito Hospitalar
New col name evolucao_Complicação Vascular
New col name evolucao_I.R.A
New col name evolucao_I.R.A Dialítica
New col name evolucao_A.V.C Hemorrágico
New col name evolucao_A.V.C Isquêmico
New col name evolucao_Combinado
New col name paciente_Nº. Paciente
New col name paciente_Idade Atual
New col name paciente_Idade no procedimento
New col name paciente_Frequência Cardíaca
New col name paciente_Frequência Respiratória
New col name paciente_Temperatura (ºC)
New col name paciente_Peso (Kg)
New col name paciente_Altura (metros)
New col name paciente_Índice de Massa Corporal
New col name paciente_Creatinina Sérica
New col name paciente_MDRD (ml/min/173m²)
New col name paciente_Clearance Cr (ml/min)
New col name paciente_Pressão Arterial_min
New col name paciente_Pressão Arterial_max
New col name

/home/titobr/mestrado/mestrado-venv/lib/python3.6/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [79]:
merge_df = pd.merge(merge_df, complicacao_agg_df,  how='left', left_on=['evolucao_Nº. Paciente','evolucao_Nº da Intervenção'], right_on = ['complicacao_Nº. Paciente','complicacao_Nº da Intervenção'])
merge_df.rename(columns=rename_columns(merge_df), inplace=True)

New col name evolucao_Nº. Paciente
New col name evolucao_Nº da Intervenção
New col name evolucao_Problema Clínico
New col name evolucao_Infarto após Intervenção
New col name evolucao_Óbito Hospitalar
New col name evolucao_Complicação Vascular
New col name evolucao_I.R.A
New col name evolucao_I.R.A Dialítica
New col name evolucao_A.V.C Hemorrágico
New col name evolucao_A.V.C Isquêmico
New col name evolucao_Combinado
New col name paciente_Nº. Paciente
New col name paciente_Idade Atual
New col name paciente_Idade no procedimento
New col name paciente_Frequência Cardíaca
New col name paciente_Frequência Respiratória
New col name paciente_Temperatura (ºC)
New col name paciente_Peso (Kg)
New col name paciente_Altura (metros)
New col name paciente_Índice de Massa Corporal
New col name paciente_Creatinina Sérica
New col name paciente_MDRD (ml/min/173m²)
New col name paciente_Clearance Cr (ml/min)
New col name paciente_Pressão Arterial_min
New col name paciente_Pressão Arterial_max
New col name

In [80]:
merge_df = pd.merge(merge_df, balao_agg_df,  how='left', left_on=['evolucao_Nº. Paciente','evolucao_Nº da Intervenção'], right_on = ['balao_Nº. Paciente','balao_Nº da Intervenção'])
merge_df.rename(columns=rename_columns(merge_df), inplace=True)

New col name evolucao_Nº. Paciente
New col name evolucao_Nº da Intervenção
New col name evolucao_Problema Clínico
New col name evolucao_Infarto após Intervenção
New col name evolucao_Óbito Hospitalar
New col name evolucao_Complicação Vascular
New col name evolucao_I.R.A
New col name evolucao_I.R.A Dialítica
New col name evolucao_A.V.C Hemorrágico
New col name evolucao_A.V.C Isquêmico
New col name evolucao_Combinado
New col name paciente_Nº. Paciente
New col name paciente_Idade Atual
New col name paciente_Idade no procedimento
New col name paciente_Frequência Cardíaca
New col name paciente_Frequência Respiratória
New col name paciente_Temperatura (ºC)
New col name paciente_Peso (Kg)
New col name paciente_Altura (metros)
New col name paciente_Índice de Massa Corporal
New col name paciente_Creatinina Sérica
New col name paciente_MDRD (ml/min/173m²)
New col name paciente_Clearance Cr (ml/min)
New col name paciente_Pressão Arterial_min
New col name paciente_Pressão Arterial_max
New col name

In [81]:
merge_df = pd.merge(merge_df, stent_agg_df,  how='left', left_on=['evolucao_Nº. Paciente','evolucao_Nº da Intervenção'], right_on = ['stent_Nº. Paciente','stent_Nº da Intervenção'])
merge_df.rename(columns=rename_columns(merge_df), inplace=True)

New col name evolucao_Nº. Paciente
New col name evolucao_Nº da Intervenção
New col name evolucao_Problema Clínico
New col name evolucao_Infarto após Intervenção
New col name evolucao_Óbito Hospitalar
New col name evolucao_Complicação Vascular
New col name evolucao_I.R.A
New col name evolucao_I.R.A Dialítica
New col name evolucao_A.V.C Hemorrágico
New col name evolucao_A.V.C Isquêmico
New col name evolucao_Combinado
New col name paciente_Nº. Paciente
New col name paciente_Idade Atual
New col name paciente_Idade no procedimento
New col name paciente_Frequência Cardíaca
New col name paciente_Frequência Respiratória
New col name paciente_Temperatura (ºC)
New col name paciente_Peso (Kg)
New col name paciente_Altura (metros)
New col name paciente_Índice de Massa Corporal
New col name paciente_Creatinina Sérica
New col name paciente_MDRD (ml/min/173m²)
New col name paciente_Clearance Cr (ml/min)
New col name paciente_Pressão Arterial_min
New col name paciente_Pressão Arterial_max
New col name

In [82]:
merge_df = pd.merge(merge_df, vaso_agg_df,  how='left', left_on=['evolucao_Nº. Paciente','evolucao_Nº da Intervenção'], right_on = ['vaso_Nº. Paciente','vaso_Nº da Intervenção'])
merge_df.rename(columns=rename_columns(merge_df), inplace=True)

New col name evolucao_Nº. Paciente
New col name evolucao_Nº da Intervenção
New col name evolucao_Problema Clínico
New col name evolucao_Infarto após Intervenção
New col name evolucao_Óbito Hospitalar
New col name evolucao_Complicação Vascular
New col name evolucao_I.R.A
New col name evolucao_I.R.A Dialítica
New col name evolucao_A.V.C Hemorrágico
New col name evolucao_A.V.C Isquêmico
New col name evolucao_Combinado
New col name paciente_Nº. Paciente
New col name paciente_Idade Atual
New col name paciente_Idade no procedimento
New col name paciente_Frequência Cardíaca
New col name paciente_Frequência Respiratória
New col name paciente_Temperatura (ºC)
New col name paciente_Peso (Kg)
New col name paciente_Altura (metros)
New col name paciente_Índice de Massa Corporal
New col name paciente_Creatinina Sérica
New col name paciente_MDRD (ml/min/173m²)
New col name paciente_Clearance Cr (ml/min)
New col name paciente_Pressão Arterial_min
New col name paciente_Pressão Arterial_max
New col name

In [83]:
merge_df.to_csv('V2_merge_df.csv')

## Split Train Test

In [84]:
paciente_testDataset_indexes = pd.read_csv('paciente_testDataset_indices.csv', index_col=0, header=None)
print(len(paciente_testDataset_indexes.index.unique()))

1750


In [85]:
test_df  = merge_df[merge_df['evolucao_Nº. Paciente'].isin(paciente_testDataset_indexes.index)]
print(len(test_df['evolucao_Nº. Paciente'].unique()))
test_df.columns

1750


Index(['evolucao_Nº. Paciente', 'evolucao_Nº da Intervenção',
       'evolucao_Problema Clínico', 'evolucao_Infarto após Intervenção',
       'evolucao_Óbito Hospitalar', 'evolucao_Complicação Vascular',
       'evolucao_I.R.A', 'evolucao_I.R.A Dialítica',
       'evolucao_A.V.C Hemorrágico', 'evolucao_A.V.C Isquêmico',
       ...
       'vaso_Ectasia - Intervenção_Sim_sum',
       'vaso_Aneurisma - Intervenção_Não_sum',
       'vaso_Aneurisma - Intervenção_Sim_sum',
       'vaso_Bifurcação - Intervenção_001_sum',
       'vaso_Bifurcação - Intervenção_010_sum',
       'vaso_Bifurcação - Intervenção_011_sum',
       'vaso_Bifurcação - Intervenção_100_sum',
       'vaso_Bifurcação - Intervenção_101_sum',
       'vaso_Bifurcação - Intervenção_110_sum',
       'vaso_Bifurcação - Intervenção_111_sum'],
      dtype='object', length=666)

In [86]:
train_df  = merge_df[~merge_df['evolucao_Nº. Paciente'].isin(paciente_testDataset_indexes.index)]
print(len(train_df['evolucao_Nº. Paciente'].unique()))
train_df.columns

6998


Index(['evolucao_Nº. Paciente', 'evolucao_Nº da Intervenção',
       'evolucao_Problema Clínico', 'evolucao_Infarto após Intervenção',
       'evolucao_Óbito Hospitalar', 'evolucao_Complicação Vascular',
       'evolucao_I.R.A', 'evolucao_I.R.A Dialítica',
       'evolucao_A.V.C Hemorrágico', 'evolucao_A.V.C Isquêmico',
       ...
       'vaso_Ectasia - Intervenção_Sim_sum',
       'vaso_Aneurisma - Intervenção_Não_sum',
       'vaso_Aneurisma - Intervenção_Sim_sum',
       'vaso_Bifurcação - Intervenção_001_sum',
       'vaso_Bifurcação - Intervenção_010_sum',
       'vaso_Bifurcação - Intervenção_011_sum',
       'vaso_Bifurcação - Intervenção_100_sum',
       'vaso_Bifurcação - Intervenção_101_sum',
       'vaso_Bifurcação - Intervenção_110_sum',
       'vaso_Bifurcação - Intervenção_111_sum'],
      dtype='object', length=666)

In [87]:
train_df.to_csv('V2_train_df.csv')
test_df.to_csv('V2_test_df.csv')

In [88]:
problema_clinico_train_df = train_df.drop(columns=[col for col in train_df.columns if ('_Nº. Paciente' in col) or ('_Nº da Intervenção' in col) or (col != 'evolucao_Problema Clínico' and 'evolucao' in col)])

In [89]:
problema_clinico_test_df = test_df.drop(columns=[col for col in test_df.columns if ('_Nº. Paciente' in col) or ('_Nº da Intervenção' in col) or (col != 'evolucao_Problema Clínico' and 'evolucao' in col)])

In [90]:
problema_clinico_train_df.to_csv('problema_clinico_V2_train_df.csv')
problema_clinico_test_df.to_csv('problema_clinico_V2_test_df.csv')